In [1]:
from tensorflow import keras
from tensorflow.keras.models import Sequential

import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, BatchNormalization

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.layers import LSTM, Dense

from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
from sklearn.model_selection import cross_val_score

In [2]:
# 데이터 불러오기
df = pd.read_csv('../data/df.csv')

In [3]:
df.head()

,동,단지명,전용면적,계약년월,층,건축년도,금리,부동산소비심리,보증금
0,165,2059,79.97,201601,4,1987,1.5,117.6,44000
1,165,2059,79.97,201601,3,1987,1.5,117.6,42000
2,165,453,133.46,201601,8,1984,1.5,117.6,80000
3,165,453,104.43,201601,6,1984,1.5,117.6,52000
4,165,453,104.43,201601,3,1984,1.5,117.6,55000


In [4]:
df.describe()

,동,단지명,전용면적,계약년월,층,건축년도,금리,부동산소비심리,보증금
count,320979.000000,320979.000000,320979.000000,320979.000000,320979.000000,320979.000000,320979.000000,320979.000000,320979.000000
mean,153.538200,2682.592419,76.076619,201711.545668,10.691213,1998.759458,1.388802,118.405118,39010.146888
std,88.406449,1679.933238,26.141417,82.104811,5.780531,10.723342,0.140256,8.987102,18613.787295
min,0.000000,0.000000,10.110000,201601.000000,0.000000,1961.000000,1.250000,93.900000,500.000000
25%,70.000000,1352.000000,59.680000,201610.000000,6.000000,1990.000000,1.250000,113.100000,26000.000000
50%,159.000000,2269.000000,80.920000,201707.000000,10.000000,1999.000000,1.500000,119.200000,37000.000000
75%,235.000000,3925.000000,84.960000,201804.000000,14.000000,2007.000000,1.500000,126.300000,50000.000000
max,331.000000,6632.000000,317.360000,201812.000000,68.000000,2018.000000,1.750000,134.100000,100000.000000


In [5]:
# 정규화하기
# MinMaxScaler 클래스 생성
scaler = MinMaxScaler()

# MinMaxScaler 클래스의 fit_transform() 메서드 사용 데이터 정규화
normalized_data = scaler.fit_transform(df)

# 정규화된 데이터, 데이터 프레임으로 다시 변환
normalized_df = pd.DataFrame(normalized_data, columns=df.columns)

In [6]:
normalized_df.head()

,동,단지명,전용면적,계약년월,층,건축년도,금리,부동산소비심리,보증금
0,0.498489,0.310464,0.227372,0.0,0.058824,0.456140,0.5,0.589552,0.437186
1,0.498489,0.310464,0.227372,0.0,0.044118,0.456140,0.5,0.589552,0.417085
2,0.498489,0.068305,0.401465,0.0,0.117647,0.403509,0.5,0.589552,0.798995
3,0.498489,0.068305,0.306981,0.0,0.088235,0.403509,0.5,0.589552,0.517588
4,0.498489,0.068305,0.306981,0.0,0.044118,0.403509,0.5,0.589552,0.547739


In [7]:
ndf = normalized_df.copy()

In [8]:
# 데이터 나누기
X = ndf.iloc[:, :-1]
y = ndf.iloc[:,[-1]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [9]:
pd.DataFrame(X_test).to_csv('../data/X_test.csv', index=False)
pd.DataFrame(y_test).to_csv('../data/y_test.csv', index=False)

In [10]:
X_test = pd.read_csv('../data/X_test.csv')
y_test = pd.read_csv('../data/y_test.csv')

In [11]:
X_test.head()

,동,단지명,전용면적,계약년월,층,건축년도,금리,부동산소비심리
0,0.906344,0.838359,0.155899,0.971564,0.058824,0.456140,0.5,0.308458
1,0.864048,0.238540,0.242086,0.023697,0.132353,0.771930,0.5,0.813433
2,0.540785,0.174306,0.432937,1.000000,0.117647,0.912281,1.0,0.000000
3,0.163142,0.091526,0.161985,0.502370,0.220588,0.947368,0.0,1.000000
4,0.432024,0.302774,0.241725,0.962085,0.088235,0.596491,0.5,0.358209


In [12]:
y_test.head()

,보증금
0,0.195980
1,0.326633
2,0.648241
3,0.547739
4,0.407035


In [13]:
print(X_train.shape)
print(y_train.shape)

(256783, 8)
(256783, 1)


In [14]:
print(X_test.shape)
print(y_test.shape)

(64196, 8)
(64196, 1)


In [15]:
model = Sequential()
model.add(LSTM(128, input_shape=(ndf.shape[1] - 1, 1)))
model.add(Dense(1))
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss='mse')

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               66560     
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 66,689
Trainable params: 66,689
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.fit(X_train, y_train, epochs=40, batch_size=32, verbose=1)

Epoch 1/40
8025/8025 [==============================] - 67s 8ms/step - loss: 0.0098
Epoch 2/40
8025/8025 [==============================] - 63s 8ms/step - loss: 0.0059
Epoch 3/40
8025/8025 [==============================] - 63s 8ms/step - loss: 0.0057
Epoch 4/40
8025/8025 [==============================] - 62s 8ms/step - loss: 0.0055
Epoch 5/40
8025/8025 [==============================] - 61s 8ms/step - loss: 0.0054
Epoch 6/40
8025/8025 [==============================] - 62s 8ms/step - loss: 0.0053
Epoch 7/40
8025/8025 [==============================] - 61s 8ms/step - loss: 0.0052
Epoch 8/40
8025/8025 [==============================] - 62s 8ms/step - loss: 0.0051
Epoch 9/40
8025/8025 [==============================] - 62s 8ms/step - loss: 0.0050
Epoch 10/40
8025/8025 [==============================] - 62s 8ms/step - loss: 0.0049
Epoch 11/40
8025/8025 [==============================] - 63s 8ms/step - loss: 0.0048
Epoch 12/40
8025/8025 [==============================] - 62s 8ms/step - lo

In [ ]:
pred = model.predict(X_test)
print(y_test.shape, pred.shape)

In [ ]:
# 실제값과 예측값을 계약년월별 평균값으로 계산
predictions = np.array(pred)
actual_values = np.array(y_test)

In [ ]:
contract_year_predictions = np.array([predictions[i:i + 12].mean() for i in range(0, len(predictions), 12)])
contract_year_actual_values = np.array([actual_values[i:i + 12].mean() for i in range(0, len(actual_values), 12)])

In [ ]:
plt.plot(contract_year_predictions, label="Predictions", linestyle='solid')
plt.plot(contract_year_actual_values, label="Actual Values")
plt.xlabel("Contract Year")
plt.ylabel("Value")
plt.legend()
plt.show()

In [ ]:
# RMSE를 계산(낮을 수록 정확함)
rmse = mean_squared_error(y_test, predictions)
print('rmse :',rmse)

# R^2을 계산(R^2가 1에 가까울수록 모델이 데이터를 잘 설명)
r2 = r2_score(y_test, predictions)
print('r2 :',r2)

# MAE를 계산(MAE가 낮을수록 예측값과 실제값이 가까워짐)
mae = mean_absolute_error(y_test, predictions)
print('mae :',mae)

# 과대적합 확인(모델 검증)

In [24]:
from sklearn.model_selection import KFold

In [23]:
'''
# 과적합되었는지 확인해보기
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.2)
''''

SyntaxError: EOL while scanning string literal (2937764013.py, line 4)

In [ ]:
'''
model.fit(X_tr, y_tr, epochs=10, batch_size=32, verbose=1) 
'''

In [ ]:
'''
y_pred = model.predict(X_val)
print(np.mean(np.abs(y_pred - y_val)))
'''

In [25]:
# K-Fold CV를 수행합니다.
kfold = KFold(n_splits=5)
cv_scores_mae = []
cv_scores_r2 = []
cv_scores_rmse = []
cv_scores = []

In [ ]:
for train_index, test_index in kfold.split(X_train):
    X_train_cv, X_test_cv = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_cv, y_test_cv = y_train.iloc[train_index], y_train.iloc[test_index]

    # 모델을 학습시킵니다.
    model.fit(X_train_cv, y_train_cv, epochs=10, batch_size=32, verbose=1)

In [ ]:
    # 모델의 성능을 평가합니다.
    y_pred = model.predict(X_test_cv)

    mse = np.mean(np.square(y_pred - y_test_cv))
    mae = np.mean(np.abs(y_pred - y_test_cv))
    r2 = r2_score(y_test_cv, y_pred)
    rmse = np.sqrt(mse)
    
    cv_scores_mae.append(mae)
    cv_scores_r2.append(r2)
    cv_scores_rmse.append(rmse)
    
    cv_scores.append([mae, r2, rmse])
    
   

In [ ]:
# K-Fold CV의 결과를 출력합니다.
cv_scores = list(cv_scores)
print(cv_scores)

In [ ]:
# 계약년월을 기준으로 그룹핑을 합니다.
df_mean = df.groupby('계약년월')['보증금'].mean()

In [ ]:
df_mean

In [ ]:
X_test['예측값'] = pred

In [ ]:
X_test.head()

In [ ]:
pd.DataFrame(X_test).to_csv('../data/X_result.csv', index=False)

In [ ]:
result = pd.read_csv('../data/X_result.csv')

In [ ]:
result.head()

In [ ]:
result = pd.concat([result, y_test], axis=1)

In [ ]:
result.head()

In [ ]:
result = result[['동', '단지명', '전용면적', '계약년월', '층', '건축년도', '금리', '부동산소비심리', '예측값', '보증금']]

In [ ]:
result.head()

In [ ]:
result = result.groupby('계약년월').mean()
result = result[['보증금', '예측값']]

In [ ]:
print(result)

In [ ]:
plt.plot(result['보증금'], label='real')
plt.plot(result['예측값'], linestyle='--', label='predict')
plt.title('predict vs real')
plt.xlabel('period')
plt.ylabel('deposit')
plt.legend()
plt.show()

In [ ]:
# 본래값 확인해보기
# 201601
print('실제값 : ',0.362120 * (100000.0-500.0)+500)
print('예측값 : ',0.362928 * (100000.0-500.0)+500)

#201602
print('실제값 : ',0.362120 * (100000.0-500.0)+500)
print('예측값 : ',0.362928 * (100000.0-500.0)+500)

# 201603
print('실제값 : ',0.344907 * (100000.0-500.0)+500)
print('예측값 : ',0.350313 * (100000.0-500.0)+500)

In [ ]:
result = pd.DataFrame({
  '계약년월': ['201601', '201602', '201603', '201604', '201605', '201606','201607', '201608', '201609','201610', '201611', '201612', 
                 '201701', '201702', '201703', '201704', '201705', '201706', '201707', '201708', '201709', '201710', '201711', '201712',
                 '201801', '201802', '201803', '201804', '201805', '201806', '201807', '201808', '201809', '201810', '201811', '201812'],
  '실제값': [0.362120 * (100000.0-500.0)+500, 0.362120 * (100000.0-500.0)+500, 0.344907 * (100000.0-500.0)+500,
              0.346538 * (100000.0-500.0)+500, 0.348332 * (100000.0-500.0)+500, 0.359644 * (100000.0-500.0)+500,
              0.359922 * (100000.0-500.0)+500, 0.349641 * (100000.0-500.0)+500, 0.375077 * (100000.0-500.0)+500,
              0.385391 * (100000.0-500.0)+500, 0.392834 * (100000.0-500.0)+500, 0.407924 * (100000.0-500.0)+500,
              0.388006 * (100000.0-500.0)+500, 0.381477 * (100000.0-500.0)+500, 0.362019 * (100000.0-500.0)+500,
              0.385883 * (100000.0-500.0)+500, 0.380351 * (100000.0-500.0)+500, 0.381554 * (100000.0-500.0)+500,
              0.382213 * (100000.0-500.0)+500, 0.376631 * (100000.0-500.0)+500, 0.389992 * (100000.0-500.0)+500,
              0.390594 * (100000.0-500.0)+500, 0.408907 * (100000.0-500.0)+500, 0.415363 * (100000.0-500.0)+500,
              0.404829 * (100000.0-500.0)+500, 0.404563 * (100000.0-500.0)+500, 0.389124 * (100000.0-500.0)+500,
              0.390689 * (100000.0-500.0)+500, 0.398179 * (100000.0-500.0)+500, 0.403312 * (100000.0-500.0)+500,
              0.395421 * (100000.0-500.0)+500, 0.396978 * (100000.0-500.0)+500, 0.403635 * (100000.0-500.0)+500,
              0.417093 * (100000.0-500.0)+500, 0.424506 * (100000.0-500.0)+500, 0.433460 * (100000.0-500.0)+500 ],
  '예측값': [0.362928 * (100000.0-500.0)+500, 0.362928 * (100000.0-500.0)+500, 0.350313 * (100000.0-500.0)+500,
              0.350392 * (100000.0-500.0)+500, 0.350796 * (100000.0-500.0)+500, 0.359717 * (100000.0-500.0)+500,
              0.359572 * (100000.0-500.0)+500, 0.354626 * (100000.0-500.0)+500, 0.369621 * (100000.0-500.0)+500,
              0.377150 * (100000.0-500.0)+500, 0.386482 * (100000.0-500.0)+500, 0.405396 * (100000.0-500.0)+500,
              0.389289 * (100000.0-500.0)+500, 0.385223 * (100000.0-500.0)+500, 0.371034 * (100000.0-500.0)+500,
              0.388146 * (100000.0-500.0)+500, 0.378188 * (100000.0-500.0)+500, 0.378080 * (100000.0-500.0)+500,
              0.375286 * (100000.0-500.0)+500, 0.373243 * (100000.0-500.0)+500, 0.377112 * (100000.0-500.0)+500,
              0.380961 * (100000.0-500.0)+500, 0.397202 * (100000.0-500.0)+500, 0.405368 * (100000.0-500.0)+500,
              0.403733 * (100000.0-500.0)+500, 0.404138 * (100000.0-500.0)+500, 0.388530 * (100000.0-500.0)+500,
              0.390369 * (100000.0-500.0)+500, 0.403761 * (100000.0-500.0)+500, 0.403241 * (100000.0-500.0)+500,
              0.396706 * (100000.0-500.0)+500, 0.395304 * (100000.0-500.0)+500, 0.394079 * (100000.0-500.0)+500,
              0.406537 * (100000.0-500.0)+500, 0.413349 * (100000.0-500.0)+500, 0.422997 * (100000.0-500.0)+500 ]
})

In [ ]:
result

In [ ]:
plt.plot(result['계약년월'], result['실제값'], label='real')
plt.plot(result['계약년월'], result['예측값'], label='predict')

plt.xlabel('period')
plt.ylabel('real')
plt.title('real vs predict')
plt.legend()

plt.show()